In [1]:
import sys
!{sys.executable} -m pip install sodapy

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sodapy import Socrata

In [3]:
client = Socrata('data.cityofnewyork.us', None)

In [4]:
def get_records(limit, offset):
    records = client.get(
        'fhrw-4uyv',
        select='borough, complaint_type',
        where='''\
            created_date >= "2017-01-01T00:00:00"
            and created_date < "2018-01-01T00:00:00"
            ''',
        order='created_date',
        limit=limit,
        offset=offset)
    
    return records

In [5]:
df = pd.DataFrame(columns=['borough', 'complaint_type'])
offset = 0; limit = 100000
prev_length = -1

In [6]:
# page through the data
while len(df) != prev_length:
    records = get_records(limit, offset)

    prev_length = len(df)
    df = pd.concat([df, pd.DataFrame.from_records(records)], ignore_index=True)
    
    offset += limit

In [7]:
# Use boolean indexing to select complaints that rank in the top ten
topTen = df['complaint_type'].value_counts().iloc[:10].index
tt = df[df['complaint_type'].isin(topTen)]

In [8]:
# The following contingency table shows the number of complaints by type for each borough, 
# for the top ten most common types of complaints
pd.crosstab(index=tt['complaint_type'], columns=tt['borough'], margins=True)

borough,BRONX,BROOKLYN,MANHATTAN,QUEENS,STATEN ISLAND,Unspecified,All
complaint_type,,,,,,,
Blocked Driveway,24574,49302,3428,54290,3465,1038,136097
HEAT/HOT WATER,68718,66984,46529,29217,2073,0,213521
Illegal Parking,16122,55380,19687,46065,7574,1294,146122
Noise,3134,15421,29002,10685,1926,3,60171
Noise - Residential,57663,67629,51026,46396,6744,694,230152
Noise - Street/Sidewalk,14025,21313,29147,7530,853,217,73085
Street Condition,11761,25432,14840,30629,10560,43,93265
Street Light Condition,18410,22458,11077,24258,6604,1388,84195
UNSANITARY CONDITION,24561,26659,14635,11474,1953,0,79282
